In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trec-2025/2025-starter-kit-main/.example.env
/kaggle/input/trec-2025/2025-starter-kit-main/main.py
/kaggle/input/trec-2025/2025-starter-kit-main/README.md
/kaggle/input/trec-2025/2025-starter-kit-main/produce_run.py
/kaggle/input/trec-2025/2025-starter-kit-main/output/tracking_data.json
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-1
/kaggle/input/trec-2025/2025-starter-kit-main/output/dragun-organizers-starter-kit-task-2
/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl
/kaggle/input/trec-2025/2025-starter-kit-main/modules/report_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/segment_retriever.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/question_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/query_generator.py
/kaggle/input/trec-2025/2025-starter-kit-main/modules/information_evaluator.py


In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth
    !pip install langchain langchain-community
    !pip install jsonlines
    !pip install --quiet langchain transformers sentencepiece

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import json
from unsloth import FastLanguageModel
from unsloth import FastModel
import torch
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template

2025-08-02 15:44:50.424514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754149490.656545      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754149490.729049      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
TOPICS_PATH = '/kaggle/input/trec-2025/2025-starter-kit-main/data/trec-2025-dragun-topics.jsonl'

def load_jsonl(fname):
    with open(fname, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
topics = load_jsonl(TOPICS_PATH)
print("Loaded", len(topics), "topics.")

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)  

from langchain.prompts import PromptTemplate

Loaded 30 topics.
==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
# import re
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline

# # 1. Enhanced Prompt Template with Examples from PolitiFact / MBFC Style
# prompt_template = PromptTemplate(
#     input_variables=["title", "body"],
#     template=(
#         "You are an expert in media literacy and critical thinking.\n"
#         "Your task is to generate 10 investigative questions that a reader should ask to evaluate the reliability, bias, and factual integrity of a news article.\n"
#         "The questions must:\n"
#         "- Be at most 300 characters.\n"
#         "- Avoid compound or overly broad questions.\n"
#         "- Address potential bias, political slant, source credibility, missing context, and factual support.\n"
#         "- Focus on identifying misinformation or slant (as PolitiFact and MBFC would).\n"
#         "- List the most important trust-checking questions first.\n\n"

#         "Example 1 (PolitiFact style):\n"
#         "Title: 'Politician Claims Crime is at All-Time High'\n"
#         "Text: A senator claims that urban crime rates have soared beyond historical levels...\n"
#         "Questions:\n"
#         "1. What official crime data is cited to support the claim?\n"
#         "2. Does the article compare the current rate with past years?\n"
#         "3. Are crime experts or analysts quoted in the article?\n"
#         "4. Does the article explain which cities are included?\n"
#         "5. Is any partisan bias evident in the language?\n"
#         "6. Are sources properly identified and verifiable?\n"
#         "7. Are dissenting expert views included?\n"
#         "8. Are statistics taken from official or third-party sites?\n"
#         "9. Are terms like 'all-time high' quantified?\n"
#         "10. Are claims linked to any political motive or bill?\n\n"

#         "Example 2 (MBFC style):\n"
#         "Title: 'Website Claims COVID-19 Vaccine Causes Infertility'\n"
#         "Text: A viral post from XYZ.com alleges the vaccine is linked to infertility...\n"
#         "Questions:\n"
#         "1. Is XYZ.com a medically reputable source?\n"
#         "2. Does the article cite scientific peer-reviewed studies?\n"
#         "3. Are claims countered by mainstream medical organizations?\n"
#         "4. Are author credentials provided?\n"
#         "5. Are experts from both sides of the issue quoted?\n"
#         "6. Are statistics cherry-picked or taken out of context?\n"
#         "7. Does the article present evidence-based rebuttals?\n"
#         "8. Are known misinformation patterns present?\n"
#         "9. Does the headline reflect the actual content?\n"
#         "10. Are misleading visuals or graphs used?\n\n"

#         "Now, given the following article, generate 10 similarly styled questions:\n\n"
#         "Title: {title}\n\n"
#         "Article text:\n{body}\n\n"
#         "Output only the questions as a numbered list:\n"
#     )
# )

# # 2. Create model pipeline
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=600,
#     temperature=0.7,
#     do_sample=True,
#     return_full_text=False
# )
# llm = HuggingFacePipeline(pipeline=pipe)
# llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")

# # 3. Regex to extract clean questions
# pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

# # 4. Extraction function
# def extract_unique_questions(raw_text):
#     seen = set()
#     questions = []
#     for line in raw_text.strip().split("\n"):
#         match = pattern.match(line.strip())
#         if match:
#             q = match.group(1).strip()
#             if 0 < len(q) <= 300 and q not in seen:
#                 seen.add(q)
#                 questions.append(q)
#     return questions

# # 5. Main inference loop
# results = []
# for idx, topic in enumerate(topics):
#     docid = topic['docid']
#     title = topic.get('title', '').strip()
#     body = topic.get('body', '').strip()[:2000]

#     if not title and not body:
#         print(f"⚠️ Skipping empty topic {docid}")
#         continue

#     questions = []
#     for attempt in range(3):
#         output = llm_chain.invoke({"title": title, "body": body})
#         text = output['text'] if isinstance(output, dict) else output
#         questions = extract_unique_questions(text)

#         if len(questions) >= 10:
#             questions = questions[:10]
#             break
#         else:
#             print(f"🔁 Retry {attempt+1} for {docid} — Got {len(questions)} questions")

#     # Fill short results with 'N/A'
#     while len(questions) < 10:
#         questions.append("N/A")

#     results.append({
#         "docid": docid,
#         "questions": questions
#     })

# # 6. Print examples
# for item in results[:3]:
#     print(f"\nDocID: {item['docid']}")
#     for i, q in enumerate(item['questions'], 1):
#         print(f"{i}. {q}")

In [6]:
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 1. Enhanced Prompt Template (TREC-compliant)
prompt_template = PromptTemplate(
    input_variables=["title", "body"],
    template=(
        "You are a media analyst tasked with creating 10 investigative questions to assess the trustworthiness of a news article.\n"
        "Each question must:\n"
        "- Be at most 300 characters\n"
        "- Focus on one issue only (no compound questions)\n"
        "- Avoid ambiguity or over-generalization\n"
        "- Help detect misinformation, bias, political slant, or credibility gaps\n\n"
        "Examples (PolitiFact/MBFC style):\n"
        "Title: 'Politician Claims Crime is at All-Time High'\n"
        "1. What data backs up the claim about crime rates?\n"
        "2. Does the article compare this rate to past years?\n"
        "...\n"
        "Title: 'Website Claims COVID-19 Vaccine Causes Infertility'\n"
        "1. Is the source medically reputable?\n"
        "2. Are scientific studies cited?\n"
        "...\n\n"
        "Now generate 10 questions for the following:\n\n"
        "Title: {title}\n\n"
        "Article text:\n{body}\n\n"
        "Output only the questions as a numbered list:\n"
    )
)

# 2. Load LLM
pipe = pipeline(
    "text-generation",
    model=model,  # assume already loaded
    tokenizer=tokenizer,  # assume already loaded
    max_new_tokens=600,
    temperature=0.7,
    do_sample=True,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)
llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")

# 3. Regex extraction
pattern = re.compile(r"^\s*\d+[.)]\s*(.+)$")

def extract_unique_questions(raw_text):
    seen = set()
    questions = []
    for line in raw_text.strip().split("\n"):
        match = pattern.match(line.strip())
        if match:
            q = match.group(1).strip()
            if 0 < len(q) <= 300 and q not in seen:
                seen.add(q)
                questions.append(q)
    return questions

# 4. Inference Loop
results = []
for topic in topics:
    docid = topic['docid']
    title = topic.get('title', '').strip()
    body = topic.get('body', '').strip()[:2000]

    if not title and not body:
        print(f"⚠️ Skipping empty topic {docid}")
        continue

    questions = []
    for attempt in range(3):
        output = llm_chain.invoke({"title": title, "body": body})
        text = output['text'] if isinstance(output, dict) else output
        questions = extract_unique_questions(text)
        if len(questions) >= 10:
            questions = questions[:10]
            break
        print(f"🔁 Retry {attempt+1} for {docid}: Got {len(questions)}")

    # Pad with 'N/A' if fewer than 10
    while len(questions) < 10:
        questions.append("N/A")

    results.append({
        "docid": docid,
        "questions": questions
    })

# 5. Print Example Output
for item in results[:3]:
    print(f"\nDocID: {item['docid']}")
    for i, q in enumerate(item['questions'], 1):
        print(f"{i}. {q}")

/tmp/ipykernel_36/3650943970.py:43: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_36/3650943970.py:44: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm, output_key="text")


🔁 Retry 1 for msmarco_v2.1_doc_34_7751734: Got 2
🔁 Retry 2 for msmarco_v2.1_doc_34_7751734: Got 1
🔁 Retry 1 for msmarco_v2.1_doc_38_227081897: Got 1
🔁 Retry 1 for msmarco_v2.1_doc_39_1165221603: Got 7
🔁 Retry 1 for msmarco_v2.1_doc_55_248024230: Got 7

DocID: msmarco_v2.1_doc_04_420132660
1. ...
2. Is there verifiable evidence that Sam Panopoulos invented the Hawaiian pizza in 1962?
3. What historical records confirm the restaurant where the pizza was first created?
4. Does the article cite sources for the claim that Hawaiian pizza is "the most insulting" in the world?
5. What evidence supports the assertion that the name "Hawaiian pizza" has no connection to Hawaii?
6. Are there documented examples of the pizza being called "Hawaiian" before its Canadian origin?
7. What sources verify the claim that pizza was a "new dish" in North America until the 1940s?
8. Does the article reference studies or surveys about the pizza's popularity or rejection in North America?
9. What historical dat

In [7]:
import pandas as pd

team_id = "CUET"
run_id = "CUET-qwen14B-v2"

submission_rows = []

for item in results:
    topic_id = item["docid"]
    for rank, question in enumerate(item["questions"], 1):
        # Clean question of unwanted tabs/newlines
        clean_q = question.replace('\t', ' ').replace('\n', ' ').strip()
        submission_rows.append([
            topic_id,
            team_id,
            run_id,
            rank,
            clean_q
        ])

# Create DataFrame and save
df = pd.DataFrame(submission_rows, columns=["topic_id", "team_id", "run_id", "rank", "question"])
df.to_csv("/kaggle/working/CUET_run7.tsv", sep="\t", index=False, header=False, encoding="utf-8")

# For me

In [8]:

# === Step 1: Basic Structure Check ===
print(f"✅ Total rows: {len(df)}")
print(f"✅ Unique topic IDs: {df['topic_id'].nunique()}")

# === Step 2: Check columns and missing data ===
if df.isnull().values.any():
    print("❌ Found missing values in the file.")
else:
    print("✅ No missing values.")

# === Step 3: Group by topic ID and check for exactly 10 questions per topic ===
topic_counts = df['topic_id'].value_counts()

incorrect_topics = topic_counts[topic_counts != 10]
if incorrect_topics.empty:
    print("✅ All topic IDs have exactly 10 questions.")
else:
    print("❌ The following topic IDs do not have exactly 10 questions:")
    print(incorrect_topics)

# === Optional: Check for duplicate questions within a topic ===
dupes_within_topic = df.duplicated(subset=["topic_id", "question"])
if dupes_within_topic.any():
    print("⚠️ Duplicate questions found within some topics.")
    duplicate_rows = df[dupes_within_topic]
    print(duplicate_rows)
else:
    print("✅ No duplicate questions within topics.")

# === Summary ===
expected_rows = df['topic_id'].nunique() * 10
if len(df) == expected_rows:
    print(f"🎯 Submission file has correct total rows: {expected_rows}")
else:
    print(f"❌ Incorrect number of total rows. Expected: {expected_rows}, Found: {len(df)}")


✅ Total rows: 300
✅ Unique topic IDs: 30
✅ No missing values.
✅ All topic IDs have exactly 10 questions.
✅ No duplicate questions within topics.
🎯 Submission file has correct total rows: 300


In [9]:
# Show duplicated questions (exact text matches)
duplicates = df[df.duplicated('question', keep=False)]
num_duplicates = len(duplicates)

print(f"🔍 Found {num_duplicates} duplicated question rows.\n")
if num_duplicates > 0:
    print("📄 Duplicated questions (full rows):")
    display(duplicates)

# Frequency count of duplicate questions
print("\n📊 Questions that appear more than once:")
duplicate_counts = df['question'].value_counts()
display(duplicate_counts[duplicate_counts > 1])

# Check for 'N/A' values (both literal string and NaN)
na_mask = df['question'].isna() | (df['question'].str.strip().str.upper() == 'N/A')
na_rows = df[na_mask]

print(f"\n⚠️ Found {len(na_rows)} rows with 'N/A' or missing question values.\n")
if len(na_rows) > 0:
    print("🚫 Rows with N/A or missing questions:")
    display(na_rows)


🔍 Found 2 duplicated question rows.

📄 Duplicated questions (full rows):


,topic_id,team_id,run_id,rank,question
0,msmarco_v2.1_doc_04_420132660,CUET,CUET-qwen14B-v2,1,...
230,msmarco_v2.1_doc_48_1289995263,CUET,CUET-qwen14B-v2,1,...



📊 Questions that appear more than once:


question
...    2
Name: count, dtype: int64


⚠️ Found 0 rows with 'N/A' or missing question values.



# For changing / regenrating Specific portion question

In [10]:
# Format: docid -> list of missing question ranks (1-indexed)
missing_questions = {
    "msmarco_v2.1_doc_04_420132660": [1],  # regenerate question 4 and 7
    "msmarco_v2.1_doc_48_1289995263": [1]     # regenerate question 2 only
}




In [11]:
patch_rows = []

for topic in topics:
    docid = topic["docid"]
    if docid not in missing_questions:
        continue

    title = topic.get('title', '').strip()
    body = topic.get('body', '').strip()[:2000]

    print(f"🔍 Regenerating for {docid}...")

    # Generate once
    for attempt in range(3):
        output = llm_chain.invoke({"title": title, "body": body})
        text = output["text"] if isinstance(output, dict) else output
        full_questions = extract_unique_questions(text)
        if len(full_questions) >= 10:
            break
        print(f"Retry {attempt+1}: Got {len(full_questions)}")

    # If not enough, pad
    while len(full_questions) < 10:
        full_questions.append("N/A")

    for rank in missing_questions[docid]:
        patch_rows.append({
            "topic_id": docid,
            "team_id": "CUET",
            "run_id": "CUET-qwen14B-v2",
            "rank": rank,
            "question": full_questions[rank - 1]  # rank is 1-based
        })


🔍 Regenerating for msmarco_v2.1_doc_04_420132660...
🔍 Regenerating for msmarco_v2.1_doc_48_1289995263...


In [13]:
import pandas as pd

# Define column names manually since submission file has no headers
col_names = ["topic_id", "team_id", "run_id", "rank", "question"]

# Step 1: Load original file (no header in TREC format)
df = pd.read_csv("/kaggle/working/CUET_run7.tsv", sep="\t", header=None, names=col_names)

# Step 2: Convert patch_rows (already in correct format) to DataFrame
patch_df = pd.DataFrame(patch_rows)

# Step 3: Remove conflicting rows in original submission
for i, row in patch_df.iterrows():
    df = df[~((df['topic_id'] == row['topic_id']) & (df['rank'] == row['rank']))]

# Step 4: Merge and sort
df_final = pd.concat([df, patch_df], ignore_index=True).sort_values(by=["topic_id", "rank"])

# Step 5: Write back with NO HEADER (TREC requirement)
df_final.to_csv("/kaggle/working/CUET_run7.tsv", sep="\t", index=False, header=False)


In [14]:
# Show duplicated questions (exact text matches)
duplicates = df[df.duplicated('question', keep=False)]
num_duplicates = len(duplicates)

print(f"🔍 Found {num_duplicates} duplicated question rows.\n")
if num_duplicates > 0:
    print("📄 Duplicated questions (full rows):")
    display(duplicates)

# Frequency count of duplicate questions
print("\n📊 Questions that appear more than once:")
duplicate_counts = df['question'].value_counts()
display(duplicate_counts[duplicate_counts > 1])

# Check for 'N/A' values (both literal string and NaN)
na_mask = df['question'].isna() | (df['question'].str.strip().str.upper() == 'N/A')
na_rows = df[na_mask]

print(f"\n⚠️ Found {len(na_rows)} rows with 'N/A' or missing question values.\n")
if len(na_rows) > 0:
    print("🚫 Rows with N/A or missing questions:")
    display(na_rows)


🔍 Found 0 duplicated question rows.


📊 Questions that appear more than once:


Series([], Name: count, dtype: int64)


⚠️ Found 0 rows with 'N/A' or missing question values.

